In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer


df = pd.read_csv('../../films_db2.csv')
df.head(10)

,title,year,id_jp,url_allo,director,country,duration,genre,first_day,first_week,...,hebdo_rank,total_spectator,copies,rating_press,rating_public,casting,budget,lang,visa,award
0,7h58 ce samedila,2007,0010478f4e067fedf4634d0a06f0f0ad,https://www.allocine.fr/film/fichefilm_gen_cfi...,sidney lumet,etatsunis,6960,thriller,8749,74208,...,7,173224,118,3.8,3.4,"[""Philip Seymour Hoffman"", ""Ethan Hawke"", ""Mar...",18000000.0,"[""anglais""]",118634.0,0.0
1,menteur menteur,1997,00cc02716a6d42f38dd2efaa773b6900,https://www.allocine.fr/film/fichefilm_gen_cfi...,tom shadyac,etatsunis,5160,comedie,64774,845332,...,1,1551253,343,-1.0,3.2,"[""Jim Carrey"", ""Maura Tierney"", ""Swoosie Kurtz...",45000000.0,"[""anglais"", ""allemand""]",92086.0,0.0
2,dream house,2011,02acc86450b3f7c7eb34ecc6c4638bea,https://www.allocine.fr/film/fichefilm_gen_cfi...,jim sheridan,etatsunis,5460,thriller,20003,152900,...,6,261717,319,1.5,2.5,"[""Daniel Craig"", ""Naomi Watts"", ""Rachel Weisz""...",55000000.0,"[""anglais""]",130951.0,0.0
3,paris,2008,02d8e008f9b224bc32c31ff34d4b90b8,https://www.allocine.fr/film/fichefilm_gen_cfi...,cedric klapisch,france,7800,comedie,140257,778154,...,1,1723642,482,3.0,3.3,"[""Juliette Binoche"", ""Romain Duris"", ""Fabrice ...",13000000.0,"[""francais""]",114879.0,0.0
4,two lovers,2008,03101f932285a244f3887c89e43f519d,https://www.allocine.fr/film/fichefilm_gen_cfi...,james gray,etatsunis,6600,drame,25189,239193,...,5,835400,229,4.4,3.7,"[""Joaquin Phoenix"", ""Gwyneth Paltrow"", ""Viness...",12000000.0,"[""anglais""]",121726.0,0.0
5,impardonnables,2011,035e407f621eee584f1e0d4741182992,https://www.allocine.fr/film/fichefilm_gen_cfi...,andre techine,france,6660,drame,12738,80970,...,12,198822,105,2.6,2.1,"[""André Dussollier"", ""Carole Bouquet"", ""Mélani...",-1.0,"[""francais""]",126143.0,0.0
6,le crime est notre affaire,2008,0525e92640a10922a71a2858b09f673c,https://www.allocine.fr/film/fichefilm_gen_cfi...,pascal thomas,france,6540,thriller,41206,314658,...,2,1219181,330,3.9,3.2,"[""Catherine Frot"", ""André Dussollier"", ""Claude...",-1.0,"[""francais""]",119480.0,2.0
7,1 chance sur 2,1998,056edc0358fb1a42599b48e24c27d430,https://www.allocine.fr/film/fichefilm_gen_cfi...,patrice leconte,france,6600,aventure action,-1,430015,...,2,1056810,536,-1.0,2.0,"[""Jean-Paul Belmondo"", ""Alain Delon"", ""Vanessa...",-1.0,"[""francais""]",-1.0,0.0
8,high fidelity,2000,06b8cfcb316029d4252298e5acf3846b,https://www.allocine.fr/film/fichefilm_gen_cfi...,stephen frears,etatsunis,6240,romance,-1,57785,...,10,169524,135,3.6,3.4,"[""John Cusack"", ""Iben Hjejle"", ""Todd Louiso"", ...",20000000.0,"[""anglais""]",100129.0,0.0
9,lame de fond,1996,071e4437064cee69815f814e65c3b655,https://www.allocine.fr/film/fichefilm_gen_cfi...,ridley scott,etatsunis,7500,aventure action,-1,44487,...,7,95515,90,-1.0,2.9,"[""Jeff Bridges"", ""John Savage"", ""Caroline Good...",38000000.0,"[""danois"", ""anglais"", ""espagnol""]",90405.0,0.0


In [2]:
# df = df.drop(["title","visa"],axis=1)
# df.head(15)

In [3]:
df.nunique()

title              2365
year                 34
id_jp              1645
url_allo           1645
director            361
country              21
duration            120
genre                20
first_day          1827
first_week         2364
first_weekend      1693
hebdo_rank           20
total_spectator    2363
copies              759
rating_press         39
rating_public        37
casting            1588
budget              245
lang                136
visa               1386
award                25
dtype: int64

In [4]:
df.isnull().sum()

title                0
year                 0
id_jp              721
url_allo           721
director             0
country              0
duration             0
genre                0
first_day            0
first_week           0
first_weekend        0
hebdo_rank           0
total_spectator      0
copies               0
rating_press       721
rating_public      721
casting            721
budget             721
lang               721
visa               721
award              721
dtype: int64

In [5]:
# df.loc[df["url_allo"].isnull()]

In [6]:
colonnes_numeriques = ["first_day","first_weekend",'budget']

df[colonnes_numeriques] = df[colonnes_numeriques].replace(-1, np.nan)

imputer = KNNImputer(n_neighbors=5)

df_imputed = imputer.fit_transform(df[colonnes_numeriques])

df[colonnes_numeriques] = df_imputed

df

,title,year,id_jp,url_allo,director,country,duration,genre,first_day,first_week,...,hebdo_rank,total_spectator,copies,rating_press,rating_public,casting,budget,lang,visa,award
0,7h58 ce samedila,2007,0010478f4e067fedf4634d0a06f0f0ad,https://www.allocine.fr/film/fichefilm_gen_cfi...,sidney lumet,etatsunis,6960,thriller,8749.000000,74208,...,7,173224,118,3.8,3.4,"[""Philip Seymour Hoffman"", ""Ethan Hawke"", ""Mar...",1.800000e+07,"[""anglais""]",118634.0,0.0
1,menteur menteur,1997,00cc02716a6d42f38dd2efaa773b6900,https://www.allocine.fr/film/fichefilm_gen_cfi...,tom shadyac,etatsunis,5160,comedie,64774.000000,845332,...,1,1551253,343,-1.0,3.2,"[""Jim Carrey"", ""Maura Tierney"", ""Swoosie Kurtz...",4.500000e+07,"[""anglais"", ""allemand""]",92086.0,0.0
2,dream house,2011,02acc86450b3f7c7eb34ecc6c4638bea,https://www.allocine.fr/film/fichefilm_gen_cfi...,jim sheridan,etatsunis,5460,thriller,20003.000000,152900,...,6,261717,319,1.5,2.5,"[""Daniel Craig"", ""Naomi Watts"", ""Rachel Weisz""...",5.500000e+07,"[""anglais""]",130951.0,0.0
3,paris,2008,02d8e008f9b224bc32c31ff34d4b90b8,https://www.allocine.fr/film/fichefilm_gen_cfi...,cedric klapisch,france,7800,comedie,140257.000000,778154,...,1,1723642,482,3.0,3.3,"[""Juliette Binoche"", ""Romain Duris"", ""Fabrice ...",1.300000e+07,"[""francais""]",114879.0,0.0
4,two lovers,2008,03101f932285a244f3887c89e43f519d,https://www.allocine.fr/film/fichefilm_gen_cfi...,james gray,etatsunis,6600,drame,25189.000000,239193,...,5,835400,229,4.4,3.7,"[""Joaquin Phoenix"", ""Gwyneth Paltrow"", ""Viness...",1.200000e+07,"[""anglais""]",121726.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2361,paycheck,2003,fc217cad46530edea35d642d6876e87e,https://www.allocine.fr/film/fichefilm_gen_cfi...,john woo,etatsunis,7080,science fiction,52746.000000,311308,...,5,689651,399,2.7,2.7,"[""Ben Affleck"", ""Uma Thurman"", ""Aaron Eckhart""...",6.000000e+07,"[""anglais""]",109777.0,0.0
2362,licorice pizza,2021,fcc1c34a7cc6b061a61cc2b20b088497,https://www.allocine.fr/film/fichefilm_gen_cfi...,paul thomas anderson,etatsunis,7980,comedie dramatique,11051.000000,116871,...,7,369833,184,4.5,3.7,"[""Alana Haim"", ""Cooper Hoffman"", ""Bradley Coop...",3.490000e+07,"[""anglais""]",156124.0,1.0
2363,after earth,2013,fd06145978961e804a966255aa2aa595,https://www.allocine.fr/film/fichefilm_gen_cfi...,m night shyamalan,etatsunis,6000,science fiction,91575.000000,548418,...,1,1276118,497,2.4,2.2,"[""Jaden Smith"", ""Will Smith"", ""Sophie Okonedo""...",1.300000e+08,"[""anglais""]",136713.0,2.0
2364,un jour sans fin,1993,fd2184dca7908ce9669fc991ec829b38,https://www.allocine.fr/film/fichefilm_gen_cfi...,harold ramis,etatsunis,6180,comedie,76773.049403,122863,...,2,521101,112,3.8,4.1,"[""Bill Murray"", ""Andie MacDowell"", ""Stephen To...",2.057385e+08,"[""anglais"", ""francais"", ""italien""]",82623.0,0.0


In [7]:
df.isnull().sum()

title                0
year                 0
id_jp              721
url_allo           721
director             0
country              0
duration             0
genre                0
first_day            0
first_week           0
first_weekend        0
hebdo_rank           0
total_spectator      0
copies               0
rating_press       721
rating_public      721
casting            721
budget               0
lang               721
visa               721
award              721
dtype: int64

In [8]:
df.nunique()

title              2365
year                 34
id_jp              1645
url_allo           1645
director            361
country              21
duration            120
genre                20
first_day          1949
first_week         2364
first_weekend      1953
hebdo_rank           20
total_spectator    2363
copies              759
rating_press         39
rating_public        37
casting            1588
budget              821
lang                136
visa               1386
award                25
dtype: int64